## 1. Crimes Data

### 1.1 Importing Crimes Data for 2021, 2022, 2023 years

In [279]:
import pandas as pd

# Загрузка данных
df_crimes_2023 = pd.read_csv('../data/delitos_2023.csv', sep=';')
df_crimes_2022 = pd.read_csv('../data/delitos_2022.csv', sep=',')
df_crimes_2021 = pd.read_excel('../data/delitos_2021.xlsx')

# Просмотр первых строк
df_crimes_2023.head(3)

,id-sum,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,1,2023,ENERO,LUNES,2/01/2023,19,Vialidad,Muertes por siniestros viales,NO,NO,BELGRANO,13,-58.445.747,-34.559.570,1
1,2,2023,ENERO,MIERCOLES,11/01/2023,11,Vialidad,Muertes por siniestros viales,NO,NO,VILLA LUGANO,8,-58.476.557,-34.673.096,1
2,3,2023,ENERO,VIERNES,13/01/2023,4,Vialidad,Muertes por siniestros viales,NO,NO,SAAVEDRA,12,-58.485.670,-34.544.011,1


In [280]:
df_crimes_2021.head(3)

,id-mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,110639,2021,octubre,MIE,2021-10-06,0.0,Hurto,Hurto total,NO,NO,VELEZ SARSFIELD,10.0,-34631129.0,-58489377.0,1
1,110640,2021,octubre,JUE,2021-10-07,23.0,Robo,Robo total,NO,NO,VELEZ SARSFIELD,10.0,-34624628.0,-58501039.0,1
2,110641,2021,octubre,MIE,2021-10-06,12.0,Robo,Robo total,SI,NO,MONTE CASTRO,10.0,-34623213.0,-58510132.0,1


In [281]:
df_crimes_2022.head(3)

,id-mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,1,2022,OCTUBRE,VIERNES,2022-10-14,3.0,Robo,Robo total,NO,NO,CHACARITA,15.0,-34.584136,-58.454704,1
1,2,2022,OCTUBRE,JUEVES,2022-10-27,5.0,Robo,Robo total,NO,NO,BARRACAS,4.0,-34.645043,-58.373194,1
2,3,2022,NOVIEMBRE,MARTES,2022-11-29,0.0,Robo,Robo total,NO,NO,CHACARITA,15.0,-34.589982,-58.446471,1


In [282]:
print(df_crimes_2021.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111962 entries, 0 to 111961
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id-mapa   111962 non-null  int64         
 1   anio      111962 non-null  int64         
 2   mes       111962 non-null  object        
 3   dia       111962 non-null  object        
 4   fecha     111962 non-null  datetime64[ns]
 5   franja    110222 non-null  float64       
 6   tipo      111962 non-null  object        
 7   subtipo   111962 non-null  object        
 8   uso_arma  111962 non-null  object        
 9   uso_moto  111962 non-null  object        
 10  barrio    109378 non-null  object        
 11  comuna    109378 non-null  float64       
 12  latitud   109383 non-null  float64       
 13  longitud  109383 non-null  float64       
 14  cantidad  111962 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(3), object(7)
memory usage: 12.8+ MB
None


In [283]:
print(df_crimes_2022.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140918 entries, 0 to 140917
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id-mapa   140918 non-null  int64  
 1   anio      140918 non-null  int64  
 2   mes       140918 non-null  object 
 3   dia       140918 non-null  object 
 4   fecha     140918 non-null  object 
 5   franja    140871 non-null  float64
 6   tipo      140918 non-null  object 
 7   subtipo   140918 non-null  object 
 8   uso_arma  140918 non-null  object 
 9   uso_moto  140918 non-null  object 
 10  barrio    138527 non-null  object 
 11  comuna    138525 non-null  float64
 12  latitud   138533 non-null  float64
 13  longitud  138533 non-null  float64
 14  cantidad  140918 non-null  int64  
dtypes: float64(4), int64(3), object(8)
memory usage: 16.1+ MB
None


In [284]:
print(df_crimes_2023.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157461 entries, 0 to 157460
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id-sum    157461 non-null  int64 
 1   anio      157461 non-null  int64 
 2   mes       157461 non-null  object
 3   dia       157461 non-null  object
 4   fecha     157461 non-null  object
 5   franja    157461 non-null  int64 
 6   tipo      157461 non-null  object
 7   subtipo   157461 non-null  object
 8   uso_arma  157461 non-null  object
 9   uso_moto  157461 non-null  object
 10  barrio    156779 non-null  object
 11  comuna    156795 non-null  object
 12  latitud   154642 non-null  object
 13  longitud  154642 non-null  object
 14  cantidad  157461 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 18.0+ MB
None


In [285]:
# Deleting unique indexs from crime datasets
df_crimes_2021 = df_crimes_2021.drop(columns={'id-mapa'})
df_crimes_2022 = df_crimes_2022.drop(columns={'id-mapa'})
df_crimes_2023 = df_crimes_2023.drop(columns={'id-sum'})
df_crimes_2023.head()

,anio,mes,dia,fecha,franja,tipo,subtipo,uso_arma,uso_moto,barrio,comuna,latitud,longitud,cantidad
0,2023,ENERO,LUNES,2/01/2023,19,Vialidad,Muertes por siniestros viales,NO,NO,BELGRANO,13,-58.445.747,-34.559.570,1
1,2023,ENERO,MIERCOLES,11/01/2023,11,Vialidad,Muertes por siniestros viales,NO,NO,VILLA LUGANO,8,-58.476.557,-34.673.096,1
2,2023,ENERO,VIERNES,13/01/2023,4,Vialidad,Muertes por siniestros viales,NO,NO,SAAVEDRA,12,-58.485.670,-34.544.011,1
3,2023,ENERO,VIERNES,13/01/2023,6,Vialidad,Muertes por siniestros viales,NO,NO,PARQUE CHACABUCO,7,-58.451.874,-34.636.364,1
4,2023,ENERO,VIERNES,13/01/2023,6,Vialidad,Muertes por siniestros viales,NO,NO,PARQUE CHACABUCO,7,-58.451.874,-34.636.364,1


In [286]:
crimes_total = pd.concat([df_crimes_2021, df_crimes_2022, df_crimes_2023], ignore_index=True)
crimes_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410341 entries, 0 to 410340
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   anio      410341 non-null  int64  
 1   mes       410341 non-null  object 
 2   dia       410341 non-null  object 
 3   fecha     410341 non-null  object 
 4   franja    408554 non-null  float64
 5   tipo      410341 non-null  object 
 6   subtipo   410341 non-null  object 
 7   uso_arma  410341 non-null  object 
 8   uso_moto  410341 non-null  object 
 9   barrio    404684 non-null  object 
 10  comuna    404698 non-null  object 
 11  latitud   402558 non-null  object 
 12  longitud  402558 non-null  object 
 13  cantidad  410341 non-null  int64  
dtypes: float64(1), int64(2), object(11)
memory usage: 43.8+ MB


### 1.2 Data Cleaning

In [287]:
crimes_total.drop(columns={'comuna'})
crimes_total.rename(columns={
    'anio' : 'year',
    'mes'  : 'month',
    'dia'  : 'day_of_week',
    'fecha': 'date',
    'franja': 'hour',
    'tipo'    : 'crime_type',
    'subtipo' : 'crime_subtype',
    'uso_arma' : 'using_arm',
    'uso_moto' : 'using_motorbike',
    'barrio': 'neighborhood',
    'latitud': 'latitude',
    'longitud': 'longitude',
    'cantidad' : 'amount'
}, inplace=True)

crimes_total.head()

,year,month,day_of_week,date,hour,crime_type,crime_subtype,using_arm,using_motorbike,neighborhood,comuna,latitude,longitude,amount
0,2021,octubre,MIE,2021-10-06 00:00:00,0.0,Hurto,Hurto total,NO,NO,VELEZ SARSFIELD,10.0,-34631129.0,-58489377.0,1
1,2021,octubre,JUE,2021-10-07 00:00:00,23.0,Robo,Robo total,NO,NO,VELEZ SARSFIELD,10.0,-34624628.0,-58501039.0,1
2,2021,octubre,MIE,2021-10-06 00:00:00,12.0,Robo,Robo total,SI,NO,MONTE CASTRO,10.0,-34623213.0,-58510132.0,1
3,2021,octubre,JUE,2021-10-07 00:00:00,20.0,Robo,Robo total,NO,NO,VELEZ SARSFIELD,10.0,-34627172.0,-58498928.0,1
4,2021,octubre,SAB,2021-10-09 00:00:00,11.0,Hurto,Hurto total,NO,NO,FLORESTA,10.0,-34626978.0,-58483313.0,1


In [288]:
crimes_total['date'] = pd.to_datetime(crimes_total['date'])
crimes_total['month'] = crimes_total['date'].dt.month
crimes_total['day_of_week'] = crimes_total['date'].dt.day_of_week
crimes_total.head()

,year,month,day_of_week,date,hour,crime_type,crime_subtype,using_arm,using_motorbike,neighborhood,comuna,latitude,longitude,amount
0,2021,10,2,2021-10-06,0.0,Hurto,Hurto total,NO,NO,VELEZ SARSFIELD,10.0,-34631129.0,-58489377.0,1
1,2021,10,3,2021-10-07,23.0,Robo,Robo total,NO,NO,VELEZ SARSFIELD,10.0,-34624628.0,-58501039.0,1
2,2021,10,2,2021-10-06,12.0,Robo,Robo total,SI,NO,MONTE CASTRO,10.0,-34623213.0,-58510132.0,1
3,2021,10,3,2021-10-07,20.0,Robo,Robo total,NO,NO,VELEZ SARSFIELD,10.0,-34627172.0,-58498928.0,1
4,2021,10,5,2021-10-09,11.0,Hurto,Hurto total,NO,NO,FLORESTA,10.0,-34626978.0,-58483313.0,1


In [289]:
bin_format = {'NO': 0, 'SI': 1}
crimes_total['using_arm'] = crimes_total['using_arm'].map(bin_format)
crimes_total['using_motorbike'] = crimes_total['using_motorbike'].map(bin_format)
crimes_total.head()

,year,month,day_of_week,date,hour,crime_type,crime_subtype,using_arm,using_motorbike,neighborhood,comuna,latitude,longitude,amount
0,2021,10,2,2021-10-06,0.0,Hurto,Hurto total,0,0,VELEZ SARSFIELD,10.0,-34631129.0,-58489377.0,1
1,2021,10,3,2021-10-07,23.0,Robo,Robo total,0,0,VELEZ SARSFIELD,10.0,-34624628.0,-58501039.0,1
2,2021,10,2,2021-10-06,12.0,Robo,Robo total,1,0,MONTE CASTRO,10.0,-34623213.0,-58510132.0,1
3,2021,10,3,2021-10-07,20.0,Robo,Robo total,0,0,VELEZ SARSFIELD,10.0,-34627172.0,-58498928.0,1
4,2021,10,5,2021-10-09,11.0,Hurto,Hurto total,0,0,FLORESTA,10.0,-34626978.0,-58483313.0,1


In [290]:
crimes_total['crime_subtype'].unique()

array(['Hurto total', 'Robo total', 'Hurto automotor', 'Robo automotor',
       'Lesiones por siniestros viales', 'Homicidios dolosos',
       'Femicidios', 'Lesiones Dolosas', 'Amenazas',
       'Muertes por siniestros viales', 'Homicidio Doloso'], dtype=object)

In [291]:
subcategory_translation = {
    'Hurto total': 'Total Theft',
    'Robo total': 'Total Robbery',
    'Hurto automotor': 'Vehicle Theft',
    'Robo automotor': 'Vehicle Robbery',
    'Lesiones por siniestros viales': 'Injuries from Traffic Accidents',
    'Homicidios dolosos': 'Intentional Homicides',
    'Femicidios': 'Femicides',
    'Lesiones Dolosas': 'Intentional Injuries',
    'Amenazas': 'Threats',
    'Muertes por siniestros viales': 'Deaths from Traffic Accidents',
    'Homicidio Doloso': 'Intentional Homicide'
}

crimes_total['crime_subtype'] = crimes_total['crime_subtype'].map(subcategory_translation)

In [292]:
crimes_total['crime_type'].unique()

array(['Hurto', 'Robo', 'Vialidad', 'Homicidios', 'Lesiones', 'Amenazas'],
      dtype=object)

In [293]:
category_translation = {
    'Hurto': 'Theft',
    'Robo': 'Robbery',
    'Vialidad': 'Traffic Incidents',
    'Homicidios': 'Homicides',
    'Lesiones': 'Injuries',
    'Amenazas': 'Threats'
}

crimes_total['crime_type'] = crimes_total['crime_type'].map(category_translation)
crimes_total.head()

,year,month,day_of_week,date,hour,crime_type,crime_subtype,using_arm,using_motorbike,neighborhood,comuna,latitude,longitude,amount
0,2021,10,2,2021-10-06,0.0,Theft,Total Theft,0,0,VELEZ SARSFIELD,10.0,-34631129.0,-58489377.0,1
1,2021,10,3,2021-10-07,23.0,Robbery,Total Robbery,0,0,VELEZ SARSFIELD,10.0,-34624628.0,-58501039.0,1
2,2021,10,2,2021-10-06,12.0,Robbery,Total Robbery,1,0,MONTE CASTRO,10.0,-34623213.0,-58510132.0,1
3,2021,10,3,2021-10-07,20.0,Robbery,Total Robbery,0,0,VELEZ SARSFIELD,10.0,-34627172.0,-58498928.0,1
4,2021,10,5,2021-10-09,11.0,Theft,Total Theft,0,0,FLORESTA,10.0,-34626978.0,-58483313.0,1


In [294]:
crimes_total['neighborhood'].unique()

array(['VELEZ SARSFIELD', 'MONTE CASTRO', 'FLORESTA', 'VILLA LURO',
       'VILLA DEL PARQUE', 'VILLA SANTA RITA', 'PALERMO', 'VILLA CRESPO',
       'VILLA SOLDATI', 'FLORES', 'VILLA GRAL. MITRE', 'RECOLETA',
       'SAN TELMO', 'CABALLITO', 'PATERNAL', 'NUEVA POMPEYA',
       'PARQUE PATRICIOS', 'SAN NICOLAS', 'ALMAGRO', 'BALVANERA',
       'AGRONOMIA', 'RETIRO', 'CONSTITUCION', 'SAAVEDRA', 'VILLA URQUIZA',
       'COGHLAN', 'BELGRANO', 'VILLA PUEYRREDON', 'SAN CRISTOBAL',
       'NUÃ‘EZ', 'VILLA ORTUZAR', 'COLEGIALES', 'VERSALLES', 'MONSERRAT',
       'CHACARITA', 'BOEDO', 'LINIERS', 'VILLA DEVOTO', 'BARRACAS',
       'VILLA LUGANO', 'PARQUE CHACABUCO', 'PUERTO MADERO', 'BOCA',
       'PARQUE AVELLANEDA', 'MATADEROS', 'VILLA RIACHUELO', 'VILLA REAL',
       'PARQUE CHAS', nan, 'NUÑEZ', 'SD', 'LA BOCA', 'CONTITUCIÓN',
       'RODRIGO BUENO', 'AV BOEDO', 'NO ESPECIFICADA',
       'GREGORIO DE LAFERRERE', 'FLORIDA', 'BERNAL', 'DOCK SUD', '0',
       'SANTA MARÍA', 'BANFIELD OESTE', 'VIL

In [295]:
crimes_total.isnull().sum()

year                  0
month                 0
day_of_week           0
date                  0
hour               1787
crime_type            0
crime_subtype         0
using_arm             0
using_motorbike       0
neighborhood       5657
comuna             5643
latitude           7783
longitude          7783
amount                0
dtype: int64

In [296]:
crimes_total = crimes_total.dropna(subset=['latitude', 'neighborhood'], how='all')
crimes_total.isnull().sum()

year                  0
month                 0
day_of_week           0
date                  0
hour               1497
crime_type            0
crime_subtype         0
using_arm             0
using_motorbike       0
neighborhood        305
comuna              297
latitude           2431
longitude          2431
amount                0
dtype: int64

In [297]:
crimes_total = crimes_total[
    ~((crimes_total['neighborhood'] == "Sin geo") & (crimes_total['latitude'].isna()))
]
crimes_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404641 entries, 0 to 410340
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   year             404641 non-null  int64         
 1   month            404641 non-null  int32         
 2   day_of_week      404641 non-null  int32         
 3   date             404641 non-null  datetime64[ns]
 4   hour             403144 non-null  float64       
 5   crime_type       404641 non-null  object        
 6   crime_subtype    404641 non-null  object        
 7   using_arm        404641 non-null  int64         
 8   using_motorbike  404641 non-null  int64         
 9   neighborhood     404336 non-null  object        
 10  comuna           404344 non-null  object        
 11  latitude         402558 non-null  object        
 12  longitude        402558 non-null  object        
 13  amount           404641 non-null  int64         
dtypes: datetime64[ns](1), flo

In [298]:
crimes_total.isnull().sum()

year                  0
month                 0
day_of_week           0
date                  0
hour               1497
crime_type            0
crime_subtype         0
using_arm             0
using_motorbike       0
neighborhood        305
comuna              297
latitude           2083
longitude          2083
amount                0
dtype: int64

In [299]:
#crimes_total.to_csv('../data/Delitos2023_cleaned.csv', index=False)


## 2. Population in each neighborhood dataset

In [300]:
df_population = pd.read_csv('../data/caba_pob_barrios_2010.csv', sep=',')
df_population.head(6)

,BARRIO,POBLACION
0,AGRONOMIA,13912
1,ALMAGRO,131699
2,BALVANERA,138926
3,BARRACAS,89452
4,BELGRANO,126267
5,BOCA,45113


In [301]:
df_population['BARRIO'].nunique()

48

In [302]:
df_population.rename(columns={
    'BARRIO': 'neighborhood',
    'POBLACION' : 'population'
}, inplace=True)

df_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   neighborhood  48 non-null     object
 1   population    48 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 900.0+ bytes


## 3. Location and geographical boundaries of the city's neighborhoods.

In [192]:
df_neighborhood = pd.read_csv('../data/barrios.csv', sep=',')

df_neighborhood.head(8)

,id,objeto,nombre,comuna,perimetro_,area_metro,geometry
0,1,BARRIO,Agronomia,15,6556.17,2122169.34,POLYGON ((-58.475888981732986 -34.591723461272...
1,2,BARRIO,Almagro,5,8537.90,4050752.25,POLYGON ((-58.416002854915654 -34.597854231564...
2,3,BARRIO,Balvanera,3,8375.82,4342280.27,POLYGON ((-58.392934155259674 -34.599636447011...
3,4,BARRIO,Barracas,4,12844.17,7954579.06,POLYGON ((-58.3706620577617 -34.62949214687238...
4,5,BARRIO,Belgrano,13,20443.29,8025458.65,POLYGON ((-58.45056826999142 -34.5356114803969...
5,6,BARRIO,Boca,4,20074.86,5036358.08,POLYGON ((-58.35125703366879 -34.6193110693859...
6,7,BARRIO,Boedo,5,6602.51,2609850.88,POLYGON ((-58.411781630067345 -34.630354566952...
7,8,BARRIO,Caballito,6,10990.96,6851028.91,POLYGON ((-58.42779700513828 -34.6220734973102...


In [193]:
df_neighborhood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          48 non-null     int64  
 1   objeto      48 non-null     object 
 2   nombre      48 non-null     object 
 3   comuna      48 non-null     int64  
 4   perimetro_  48 non-null     float64
 5   area_metro  48 non-null     float64
 6   geometry    48 non-null     object 
dtypes: float64(2), int64(2), object(3)
memory usage: 2.8+ KB


In [194]:
df_neighborhood['objeto'].unique()

array(['BARRIO'], dtype=object)

In [195]:
df_neighborhood = df_neighborhood.drop(columns={'objeto', 'id'})
df_neighborhood.rename(columns={
    'nombre': 'neighborhood',
    'comuna' : 'commune',
    'perimetro_' : 'perimeter_neib',
    'area_metro' : 'area_neib'
}, inplace=True)
df_neighborhood.head()

,neighborhood,commune,perimeter_neib,area_neib,geometry
0,Agronomia,15,6556.17,2122169.34,POLYGON ((-58.475888981732986 -34.591723461272...
1,Almagro,5,8537.90,4050752.25,POLYGON ((-58.416002854915654 -34.597854231564...
2,Balvanera,3,8375.82,4342280.27,POLYGON ((-58.392934155259674 -34.599636447011...
3,Barracas,4,12844.17,7954579.06,POLYGON ((-58.3706620577617 -34.62949214687238...
4,Belgrano,13,20443.29,8025458.65,POLYGON ((-58.45056826999142 -34.5356114803969...


In [196]:
df_neighborhood['neighborhood'] = df_neighborhood['neighborhood'].str.upper()

In [197]:
df_neighborhood['geometry'][1]

'POLYGON ((-58.416002854915654 -34.59785423156421, -58.415693821186316 -34.59785881306559, -58.41569310064061 -34.59785882374696, -58.415665646742056 -34.59785923027451, -58.4156649272858 -34.597859240927875, -58.41463026542625 -34.59789742434461, -58.413956512305795 -34.59792230600732, -58.413923180197884 -34.597923536863135, -58.41325503088718 -34.5979482077055, -58.412986336280575 -34.59795809669334, -58.41209347069164 -34.59799307817072, -58.412048595967626 -34.59799553824341, -58.411989708858094 -34.597998765828905, -58.41192135079324 -34.598002513125124, -58.41212656688081 -34.599295577317605, -58.41225653678926 -34.60011409533679, -58.41228141171682 -34.60024290997449, -58.41230247168985 -34.600351511319474, -58.412594078178884 -34.60160123026931, -58.41280644277982 -34.602511050583104, -58.41297514972213 -34.603233877671194, -58.413185908917086 -34.604136864545964, -58.41322438650256 -34.60430160254282, -58.41323125865553 -34.60433106743, -58.41341476728344 -34.605118381160466,

## 4. Geographically referenced list of public and privately managed establishments. Map display is not possible due to the coordinate format.

In [270]:
df_education = pd.read_csv('../data/establecimientos_educativos_WGS84.csv', sep=';')

df_education.head(3)

,WKT,id,cui,cueanexo,cue,anexo,sector,dom_edific,dom_establ,nombre_est,...,depfun_otr,de,comuna,barrio,area_progr,estado,longitud,latitud,Unnamed: 29,Unnamed: 30
0,MULTIPOINT ((-58.4280631417123 -34.66077257316...,1,200852,20153600,201536,0,1,Carlos Berg 3460,Carlos Berg 3460,DR. JOSE BENJAMIN ZUBIAUR,...,1,19,8.0,Villa Soldati,05 - HOSP PENNA,1,103.229.933.400,96.504.748.620,NaN,NaN
1,MULTIPOINT ((-58.4328765952925 -34.64814147013...,2,200871,20227000,202270,0,1,AgustÃ­n de Vedia 2519,Cnel. Martiniano Chilavert 1935,EI NÂº 06 DE 19,...,1,19,7.0,Flores,06 - HOSP PIÃ‘ERO,1,102.789.131.100,97.906.111.620,NaN,NaN
2,MULTIPOINT ((-58.4087767199078 -34.58565084604...,3,200014,20064600,200646,0,1,Juncal 3131,Juncal 3131,WENCESLAO POSSE,...,1,1,14.0,Palermo,04 - HOSP FERNANDEZ,1,105.002.246.800,104.837.501.600,NaN,NaN


In [271]:
df_education = df_education.drop(columns={'id', 'cui', 'cueanexo', 'cue', 'anexo', 'telefono', 'email', 'codpost', 'estado',
                           'web_megcba', 'nombre_est', 'area_progr', 'de', 'dom_edific', 'dom_establ', 'depfun', 'depfun_otr',
                           'nivel', 'nivelmodal', 'Unnamed: 29', 'Unnamed: 30', 'longitud', 'latitud'})
df_education.head()

,WKT,sector,nombre_abr,nivmod,tipest,tipest_abr,comuna,barrio
0,MULTIPOINT ((-58.4280631417123 -34.66077257316...,1,EPjs 09,PriCom,Escuela de Jornada Simple,JS,8.0,Villa Soldati
1,MULTIPOINT ((-58.4328765952925 -34.64814147013...,1,EI 06/19,IniCom,Escuela Infantil,EI,7.0,Flores
2,MULTIPOINT ((-58.4087767199078 -34.58565084604...,1,EPjs 16,PriCom,Escuela de Jornada Simple,JS,14.0,Palermo
3,MULTIPOINT ((-58.4328765952925 -34.64814147013...,1,EEM 03/19,SecCom,Escuela de EducaciÃ³n Media,EEM,7.0,Flores
4,MULTIPOINT ((-58.4614370361421 -34.62830726222...,1,EPjc 02,PriCom,Escuela de Jornada Completa,JC,7.0,Flores


In [272]:
df_education['WKT'][1]

'MULTIPOINT ((-58.4328765952925 -34.6481414701348))'

In [273]:
import re

def extract_coords(geom_str):
    match = re.search(r'\(\((-?\d+\.\d+)\s+(-?\d+\.\d+)\)\)', geom_str)
    if match:
        lon = float(match.group(1))
        lat = float(match.group(2))
        return lon, lat
    return None, None

df_education[['institution_long', 'institution_lat']] = df_education['WKT'].apply(
    lambda s: pd.Series(extract_coords(s))
)
df_education = df_education.drop(columns={'WKT', 'tipest', 'tipest_abr', 'comuna'})
df_education.head()

,sector,nombre_abr,nivmod,barrio,institution_long,institution_lat
0,1,EPjs 09,PriCom,Villa Soldati,-58.428063,-34.660773
1,1,EI 06/19,IniCom,Flores,-58.432877,-34.648141
2,1,EPjs 16,PriCom,Palermo,-58.408777,-34.585651
3,1,EEM 03/19,SecCom,Flores,-58.432877,-34.648141
4,1,EPjc 02,PriCom,Flores,-58.461437,-34.628307


In [274]:
df_education.rename(columns={
    'barrio': 'neighborhood',
    'nivmod' : 'institution_level',
    'nombre_abr' : 'institution_name'
}, inplace=True)

df_education['neighborhood'] = df_education['neighborhood'].str.upper()
df_education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3045 entries, 0 to 3044
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sector             3045 non-null   int64  
 1   institution_name   3045 non-null   object 
 2   institution_level  3044 non-null   object 
 3   neighborhood       3044 non-null   object 
 4   institution_long   3042 non-null   float64
 5   institution_lat    3042 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 142.9+ KB


In [275]:
df_education.isna().sum()

sector               0
institution_name     0
institution_level    1
neighborhood         1
institution_long     3
institution_lat      3
dtype: int64

In [276]:
df_education = df_education.dropna(subset=['institution_lat', 'neighborhood'], how='all')
df_education.isnull().sum()

sector               0
institution_name     0
institution_level    1
neighborhood         1
institution_long     3
institution_lat      3
dtype: int64

In [277]:
df_education.head()

,sector,institution_name,institution_level,neighborhood,institution_long,institution_lat
0,1,EPjs 09,PriCom,VILLA SOLDATI,-58.428063,-34.660773
1,1,EI 06/19,IniCom,FLORES,-58.432877,-34.648141
2,1,EPjs 16,PriCom,PALERMO,-58.408777,-34.585651
3,1,EEM 03/19,SecCom,FLORES,-58.432877,-34.648141
4,1,EPjc 02,PriCom,FLORES,-58.461437,-34.628307


## 5. Geographic location of the hospitals in the City of Buenos Aires.

In [344]:
df_hospitals = pd.read_csv('../data/hospitales.csv', sep=',')

df_hospitals.head(3)

,fna,gna,gna_sym,nam,esp,ate,dir,bar,com,tel,fax,web,sag,geometry
0,Hospital de Pediatria Dr. J. Garrahan,Hospital de Niños,Hospital de Ninios,Dr. J. Garrahan,Pediatria,Atención Ambulatoria - Diagnóstico - Internaci...,Combate De Los Pozos 1881,Parque Patricios,4,4941-8772 / 4942-7475 | Guardia: 4941-8702,4941-8532,www.garrahan.gov.ar,Ministerio de Salud,POINT (26549.56742604346 69922.54409977954)
1,Hospital Especializado de Emerg Psiquiatricas ...,Hospital Especializado,Hospital Especializado,Emerg Psiquiatricas Torcuato de Alvear,Salud Mental,Atención Ambulatoria - Diagnóstico - Internación,Warnes Av. 2630,Paternal,15,4521-0273 / 0090 / 0983 | Guardia: 4521-5555 /...,4521-2537,https://buenosaires.gob.ar/salud/hospitales-y-...,Ministerio de Salud,POINT (18903.751962686063 73556.13379543225)
2,Hospital Especializado de Gastroenterologia B....,Hospital Especializado,Hospital Especializado,Gastroenterologia B. Udaondo,Enf. Apar. Digestivo,Atención Ambulatoria - Diagnóstico - Internaci...,Caseros Av. 2061,Parque Patricios,4,4306-4641 al 4649 | Guardia: 4306-4641 al 49 (...,4306-3013,https://buenosaires.gob.ar/salud/hospitales-y-...,Ministerio de Salud,POINT (26601.295009385656 69455.68447203121)


In [345]:
df_hospitals = df_hospitals.drop(columns={'tel', 'com', 'fax', 'web', 'nam', 'esp', 'dir', 'gna', 'ate', 'sag', 'geometry', 'fna'})
df_hospitals.head()

,gna_sym,bar
0,Hospital de Ninios,Parque Patricios
1,Hospital Especializado,Paternal
2,Hospital Especializado,Parque Patricios
3,Hospital Especializado,Parque Patricios
4,Hospital Especializado,Almagro


In [346]:
df_hospitals.rename(columns={
    'bar': 'neighborhood',
    'gna_sym' : 'hospital_type'
}, inplace=True)

df_hospitals['neighborhood'] = df_hospitals['neighborhood'].str.upper()
df_hospitals = df_hospitals.groupby(['neighborhood'], as_index=False)['hospital_type'].count()

df_hospitals.rename(columns={
    'size': 'hospital_amount'
}, inplace=True)
df_hospitals

,neighborhood,hospital_type
0,ALMAGRO,1
1,BALVANERA,1
2,BARRACAS,6
3,BELGRANO,1
4,BOCA,2
5,CABALLITO,4
6,COGHLAN,1
7,FLORES,2
8,LINIERS,1
9,MONTE CASTRO,2


## 6. Green Zones

In [209]:
df_green_spase = pd.read_csv('../data/espacio_verde_publico.csv', sep=',')

df_green_spase.head(3)

,id,nombre,nom_mapa,barrio,comuna,ubicacion,clasificac,tiene_pati,apadrinada,decreto,fecha_decr,ordenanza_,fecha_orde,boletin_of,fecha_bole,area,perimetro,observacio,geometry
0,1442,NaN,NaN,Villa Urquiza,12,NaN,CANTERO CENTRAL,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162.332974,185.793518,Digitalizado por USIG en Agosto de 2015 - Cant...,POLYGON ((-58.49150207266336 -34.5759953540672...
1,1443,NaN,NaN,Villa Urquiza,12,NaN,CANTERO CENTRAL,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166.593536,183.701278,Digitalizado por USIG en Agosto de 2015 - Cant...,POLYGON ((-58.490401619722974 -34.575387900733...
2,1444,Jardín Japonés,Jardín Japonés,Palermo,14,"DEL LIBERTADOR, AV. - SARMIENTO, AV. - BERRO, ...",PLAZA,SI,NO,2.833,30/11/1989,43.927,1989-09-11,18.679,1989-12-13,13216.567085,3283.857360,DIFERENTES NOMBRES - Redigitalización en Agost...,MULTIPOLYGON (((-58.40932624744171 -34.5756881...


In [210]:
df_green_spase = df_green_spase.drop(columns={'nombre', 'nom_mapa', 'ubicacion', 'clasificac', 'apadrinada', 'decreto', 'fecha_decr', 'ordenanza_', 
                                              'fecha_orde', 'boletin_of', 'fecha_bole', 'observacio'})

df_green_spase.rename(columns={
    'barrio': 'neighborhood',
    'comuna' : 'commune',
    'perimetro' : 'zone_perimeter',
    'area' : 'zone_area'
}, inplace=True)

df_green_spase['neighborhood'] = df_green_spase['neighborhood'].str.upper()
df_green_spase.head()

,id,neighborhood,commune,tiene_pati,zone_area,zone_perimeter,geometry
0,1442,VILLA URQUIZA,12,NO,162.332974,185.793518,POLYGON ((-58.49150207266336 -34.5759953540672...
1,1443,VILLA URQUIZA,12,NO,166.593536,183.701278,POLYGON ((-58.490401619722974 -34.575387900733...
2,1444,PALERMO,14,SI,13216.567085,3283.857360,MULTIPOLYGON (((-58.40932624744171 -34.5756881...
3,1445,PALERMO,14,SI,96633.332061,8260.718105,MULTIPOLYGON (((-58.41118016551363 -34.5757493...
4,1446,VILLA SOLDATI,8,NO,337.704941,399.457430,POLYGON ((-58.454623092075494 -34.655655651825...


In [211]:
df_green_spase['tiene_pati'] = df_green_spase['tiene_pati'].map(bin_format)

In [212]:
df_green_spase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2144 entries, 0 to 2143
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2144 non-null   int64  
 1   neighborhood    2144 non-null   object 
 2   commune         2144 non-null   int64  
 3   tiene_pati      2124 non-null   float64
 4   zone_area       2144 non-null   float64
 5   zone_perimeter  2144 non-null   float64
 6   geometry        2144 non-null   object 
dtypes: float64(3), int64(2), object(2)
memory usage: 117.4+ KB


## 7. Subway

In [218]:
df_subway = pd.read_excel('../data/estaciones-de-subte.xlsx')
df_subway.head(4)

,long,lat,id,estacion,linea
0,-58.398928,-34.635750,1,CASEROS,H
1,-58.400970,-34.629376,2,INCLAN - MEZQUITA AL AHMAD,H
2,-58.402323,-34.623092,3,HUMBERTO 1Â°,H
3,-58.404732,-34.615242,4,VENEZUELA,H


In [219]:
df_subway.rename(columns={
    'long': 'long_station',
    'lat' : 'lat_station',
    'linea' : 'line',
    'estacion' : 'station'
}, inplace=True)

In [220]:
df_subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   long_station  90 non-null     float64
 1   lat_station   90 non-null     float64
 2   id            90 non-null     int64  
 3   station       90 non-null     object 
 4   line          90 non-null     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 3.6+ KB


## 8. Presence of slums in the area of ​​the city

In [241]:
url = "https://de.wikipedia.org/wiki/Liste_der_informellen_Siedlungen_in_Buenos_Aires"

tables = pd.read_html(url)

df_slums = tables[0]
df_slums.head()

,Name,Begrenzung,Stadtteil
0,Bajo AU7,"Parque de La Ciudad, Avenida Roca, Avenida Lac...",Villa Soldati
1,Calacita,"Barros Pazos, Lacarra, B.y Ordoñez, Laguna",Villa Soldati
2,Carrilo 2,"Mariano Acosta, Castañares, Lacarra",Villa Soldati
3,Piletones,"Lacarra, B. Pazos, Parque Indoamericano, Lago ...",Villa Soldati
4,Villa 1-11-14,"Avenida P. Moreno, Varela, Club DAOM, Riestra,...",Flores


In [242]:
df_slums.rename(columns={
    'Stadtteil' : 'neighborhood',
    'Name' : 'slum'
}, inplace=True)
df_slums = df_slums.drop(columns={"Begrenzung"})
df_slums['neighborhood'] = df_slums['neighborhood'].str.upper()
df_slums.head()

,slum,neighborhood
0,Bajo AU7,VILLA SOLDATI
1,Calacita,VILLA SOLDATI
2,Carrilo 2,VILLA SOLDATI
3,Piletones,VILLA SOLDATI
4,Villa 1-11-14,FLORES


In [243]:
slums_count = df_slums.groupby('neighborhood', as_index=False)['slum'].count()
slums_count

,neighborhood,slum
0,BARRACAS,2
1,FLORES,1
2,NUEVA POMPEYA,1
3,PARQUE AVELLANEDA,1
4,PARQUE CHACABUCO,1
5,PUERTO MADERO,1
6,RETIRO,2
7,VILLA LUGANO,5
8,VILLA RIACHUELO,1
9,VILLA SOLDATI,5


## 9. Noise Map

In [261]:
df_noise_map_afternoon = pd.read_csv('../data/mapa_de_ruido_diurno.csv', sep = ',')
df_noise_map_night = pd.read_csv('../data/mapa_de_ruido_nocturno.csv', sep = ',')
df_noise_map_afternoon.head()

,wkt,limite_inferior_rango_db,limite_superior_rango_db,comuna,leyenda,rango_db,color,periodo
0,MULTIPOLYGON (((-58.3714231269834 -34.57855223...,30.0,35.0,1,35 dBA o menos,30-35,169 255 115,Diurno
1,MULTIPOLYGON (((-58.3714231269834 -34.57855223...,35.0,40.0,1,35-40 dBA,35-40,76 230 0,Diurno
2,MULTIPOLYGON (((-58.3714231117057 -34.57856621...,40.0,45.0,1,40-45 dBA,40-45,38 115 0,Diurno
3,MULTIPOLYGON (((-58.3719240383553 -34.57857520...,45.0,50.0,1,45-50 dBA,45-50,255 255 0,Diurno
4,MULTIPOLYGON (((-58.3726218651739 -34.57866586...,50.0,55.0,1,50-55 dBA,50-55,254 204 92,Diurno


In [262]:
df_noise_map_night.head()

,wkt,limite_inferior_rango_db,limite_superior_rango_db,comuna,leyenda,rango_db,color,periodo
0,MULTIPOLYGON (((-58.3714231195791 -34.57855900...,30.0,35.0,1,35 dBA o menos,30-35,169 255 115,Nocturno
1,MULTIPOLYGON (((-58.3714231195791 -34.57855900...,35.0,40.0,1,35 dBA o menos,35-40,169 255 115,Nocturno
2,MULTIPOLYGON (((-58.3714231039585 -34.57857329...,40.0,45.0,1,40-45 dBA,40-45,38 115 0,Nocturno
3,MULTIPOLYGON (((-58.3723028127677 -34.57857548...,45.0,50.0,1,45-50 dBA,45-50,255 255 0,Nocturno
4,MULTIPOLYGON (((-58.372621821452 -34.578706396...,50.0,55.0,1,50-55 dBA,50-55,254 204 92,Nocturno


In [263]:
df_noise_map = pd.concat([df_noise_map_afternoon, df_noise_map_night], ignore_index=True)
df_noise_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   wkt                       316 non-null    object 
 1   limite_inferior_rango_db  316 non-null    float64
 2   limite_superior_rango_db  316 non-null    float64
 3   comuna                    316 non-null    int64  
 4   leyenda                   316 non-null    object 
 5   rango_db                  316 non-null    object 
 6   color                     316 non-null    object 
 7   periodo                   316 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 19.9+ KB


In [264]:
df_noise_map['range_db_mean'] = (df_noise_map['limite_inferior_rango_db'] + df_noise_map['limite_superior_rango_db'])/2
df_noise_map = df_noise_map.drop(columns={'limite_inferior_rango_db', 'limite_superior_rango_db', 'leyenda', 'color'})
df_noise_map.tail()

,wkt,comuna,rango_db,periodo,range_db_mean
311,MULTIPOLYGON (((-58.4671399189865 -34.57263816...,15,65-70,Nocturno,67.5
312,MULTIPOLYGON (((-58.4671399189865 -34.57263816...,15,70-75,Nocturno,72.5
313,MULTIPOLYGON (((-58.4636210010632 -34.57484398...,15,75-80,Nocturno,77.5
314,MULTIPOLYGON (((-58.4777986500077 -34.59979550...,15,80-85,Nocturno,82.5
315,MULTIPOLYGON (((-58.477017671962 -34.599972358...,15,85-90,Nocturno,87.5


In [265]:
trans_period = {"Nocturno" : "Night", "Diurno" : "Day"}
df_noise_map['periodo'] = df_noise_map['periodo'].map(trans_period)

In [266]:
df_noise_map.drop(columns={"comuna"})
df_noise_map.rename(columns={
    'wkt': 'noise_coord',
    'rango_db' : 'range_db',
    'periodo' : 'period'
}, inplace=True)
df_noise_map.head()

,noise_coord,comuna,range_db,period,range_db_mean
0,MULTIPOLYGON (((-58.3714231269834 -34.57855223...,1,30-35,Day,32.5
1,MULTIPOLYGON (((-58.3714231269834 -34.57855223...,1,35-40,Day,37.5
2,MULTIPOLYGON (((-58.3714231117057 -34.57856621...,1,40-45,Day,42.5
3,MULTIPOLYGON (((-58.3719240383553 -34.57857520...,1,45-50,Day,47.5
4,MULTIPOLYGON (((-58.3726218651739 -34.57866586...,1,50-55,Day,52.5


In [244]:
df_noise_map['noise_coord'][1]

'MULTIPOLYGON (((-58.3714231269834 -34.5785522354354,-58.3714231117057 -34.5785662114393,-58.3715186303293 -34.5784873496149,-58.3715031942322 -34.5784861314365,-58.3714231269834 -34.5785522354354)),((-58.3680007150571 -34.5803810166556,-58.368016357524 -34.5803822470819,-58.3680249175468 -34.5803751346279,-58.3680078094458 -34.5803751213562,-58.3680007150571 -34.5803810166556)),((-58.3663415579204 -34.581519098207,-58.36635553478 -34.581520192778,-58.3663554890484 -34.5815201539798,-58.3663414795217 -34.581519033241,-58.3663415579204 -34.581519098207)),((-58.3629585122399 -34.5843490768682,-58.3630811965881 -34.5843584718666,-58.3630914890971 -34.5843592852575,-58.3630908931303 -34.5843572610274,-58.3629569659097 -34.5843469950697,-58.3629585122399 -34.5843490768682)),((-58.3637498055859 -34.5858588260094,-58.3637059127381 -34.5862419359195,-58.3634508823178 -34.5862222287402,-58.3634506060135 -34.5862226837451,-58.3636144232073 -34.5862354047821,-58.3636260430741 -34.5862364289721,-5

## Neib

In [365]:
import geopandas as gpd
from shapely import wkt

# Районы Буэнос-Айреса (границы)
barrios = gpd.read_file("https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-educacion/barrios/barrios.geojson")  # или barrios.shp
barrios = barrios.to_crs(epsg=4326)

In [366]:
barrios = barrios.drop(columns={'objeto', 'id'})
barrios.rename(columns={
    'nombre': 'neighborhood',
    'comuna' : 'commune',
    'perimetro_' : 'perimeter_neib',
    'area_metro' : 'area_neib'
}, inplace=True)
barrios['neighborhood'] = barrios['neighborhood'].str.upper()
barrios.head()

,neighborhood,commune,perimeter_neib,area_neib,geometry
0,AGRONOMIA,15,6556.17,2122169.34,"POLYGON ((-58.47589 -34.59172, -58.47598 -34.5..."
1,ALMAGRO,5,8537.90,4050752.25,"POLYGON ((-58.416 -34.59785, -58.41569 -34.597..."
2,BALVANERA,3,8375.82,4342280.27,"POLYGON ((-58.39293 -34.59964, -58.39277 -34.6..."
3,BARRACAS,4,12844.17,7954579.06,"POLYGON ((-58.37066 -34.62949, -58.37064 -34.6..."
4,BELGRANO,13,20443.29,8025458.65,"POLYGON ((-58.45057 -34.53561, -58.45066 -34.5..."


In [367]:
geometry = wkt.loads(df_noise_map['noise_coord'][1])

gdf = gpd.GeoDataFrame(geometry=[geometry], crs="EPSG:4326")

intersecting = barrios[barrios.intersects(geometry)]

containing = barrios[barrios.contains(geometry)]
intersecting

,neighborhood,commune,perimeter_neib,area_neib,geometry
5,BOCA,4,20074.86,5036358.08,"POLYGON ((-58.35126 -34.61931, -58.35126 -34.6..."
26,PUERTO MADERO,1,17697.33,5029894.60,"POLYGON ((-58.35126 -34.61931, -58.35127 -34.6..."
28,RETIRO,1,18817.35,4512826.99,"POLYGON ((-58.35649 -34.59503, -58.35711 -34.5..."


In [368]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_subway['long_station'], df_subway['lat_station'])]
gdf_subway = gpd.GeoDataFrame(df_subway, geometry=geometry, crs='EPSG:4326')  # WGS84

# Шаг 2: Убедимся, что районы тоже в той же проекции
barrios = barrios.to_crs('EPSG:4326')

# Шаг 3: Spatial Join
gdf_subway_with_neigh = gpd.sjoin(gdf_subway, barrios[['neighborhood', 'geometry']], how='left', predicate='within')

In [369]:
gdf_subway_with_neigh = gdf_subway_with_neigh.drop(columns={'long_station', 'lat_station', 'geometry'})
gdf_subway_with_neigh.head()

,id,station,line,index_right,neighborhood
0,1,CASEROS,H,24,PARQUE PATRICIOS
1,2,INCLAN - MEZQUITA AL AHMAD,H,24,PARQUE PATRICIOS
2,3,HUMBERTO 1Â°,H,30,SAN CRISTOBAL
3,4,VENEZUELA,H,2,BALVANERA
4,5,ONCE - 30 DE DICIEMBRE,H,2,BALVANERA


In [370]:
gdf_subway_with_neigh = gdf_subway_with_neigh.groupby('neighborhood', as_index=False)['station'].count()
gdf_subway_with_neigh.rename(columns={'station' : 'subway_stations_amount'})

,neighborhood,subway_stations_amount
0,ALMAGRO,5
1,BALVANERA,12
2,BELGRANO,2
3,BOEDO,1
4,CABALLITO,4
5,CHACARITA,2
6,COLEGIALES,1
7,CONSTITUCION,5
8,FLORES,5
9,MONSERRAT,9


In [375]:
df_noise_map.shape

(316, 6)

In [378]:
# Шаг 1 — преобразуем строку в shapely-геометрию
df_noise_map['geometry'] = df_noise_map['noise_coord'].apply(wkt.loads)

# Шаг 2 — создаём GeoDataFrame
gdf_noise = gpd.GeoDataFrame(df_noise_map.drop(columns='noise_coord'), geometry='geometry', crs='EPSG:4326')

# Преобразуем проекцию в метры (для площади)
gdf_noise = gdf_noise.to_crs(epsg=3857)
barrios = barrios.to_crs(epsg=3857)

# Шаг 3 — spatial intersection: пересечение районов и зон шума
noise_inter_area = gpd.overlay(gdf_noise, barrios[['neighborhood', 'area_neib', 'geometry']], how='intersection')

# Шаг 4 — вычисляем площадь пересечения
noise_inter_area['inter_area'] = noise_inter_area.geometry.area
noise_inter_area.head(19)

,comuna,range_db,period,range_db_mean,neighborhood,area_neib,geometry,inter_area
0,1,30-35,Day,32.5,BOCA,5036358.08,"POLYGON ((-6496045.176 -4112296.739, -6496045....",0.014662
1,1,35-40,Day,37.5,BOCA,5036358.08,"POLYGON ((-6496050.419 -4112297.358, -6496045....",0.062779
2,1,60-65,Day,62.5,BOCA,5036358.08,"POLYGON ((-6496173.78 -4112058.387, -6496176.6...",101.820034
3,1,65-70,Day,67.5,BOCA,5036358.08,"MULTIPOLYGON (((-6496173.78 -4112058.387, -649...",287.938080
4,1,70-75,Day,72.5,BOCA,5036358.08,"MULTIPOLYGON (((-6496290.643 -4112203.143, -64...",425.013803
5,1,75-80,Day,77.5,BOCA,5036358.08,"MULTIPOLYGON (((-6496359.403 -4112292.202, -64...",3235.464450
6,1,80-85,Day,82.5,BOCA,5036358.08,"MULTIPOLYGON (((-6497323.813 -4113062.51, -649...",385.566896
7,4,30-35,Day,32.5,BOCA,5036358.08,"MULTIPOLYGON (((-6496045.176 -4112296.739, -64...",592.629903
8,4,35-40,Day,37.5,BOCA,5036358.08,"MULTIPOLYGON (((-6496052.704 -4112297.687, -64...",1937.702285
9,4,40-45,Day,42.5,BOCA,5036358.08,"MULTIPOLYGON (((-6496057.919 -4112298.305, -64...",4805.851987


In [381]:
noise_inter_area['mea_db_per_area'] = noise_inter_area['range_db_mean']*noise_inter_area['inter_area']/noise_inter_area['area_neib']
noise_inter_area_clean = noise_inter_area.drop(columns={'range_db', 'comuna', 'range_db_mean','inter_area'})
noise_inter_area_clean = noise_inter_area_clean.groupby(['neighborhood', 'period'], as_index=False)['mea_db_per_area'].sum()
noise_inter_area_clean.head()

,neighborhood,period,mea_db_per_area
0,AGRONOMIA,Day,22.211583
1,AGRONOMIA,Night,20.322316
2,ALMAGRO,Day,28.458668
3,ALMAGRO,Night,26.236751
4,BALVANERA,Day,27.401132


In [383]:
noise_inter_area_pivot = noise_inter_area_clean.pivot(index='neighborhood', columns='period', values='mea_db_per_area').reset_index()
noise_inter_area_pivot.columns = ['neighborhood', 'day_noise', 'night_noise']
noise_inter_area_pivot.head()

,neighborhood,day_noise,night_noise
0,AGRONOMIA,22.211583,20.322316
1,ALMAGRO,28.458668,26.236751
2,BALVANERA,27.401132,25.515022
3,BARRACAS,32.513541,30.952629
4,BELGRANO,26.384386,24.983233


In [385]:
crimes_total.head()

,year,month,day_of_week,date,hour,crime_type,crime_subtype,using_arm,using_motorbike,neighborhood,comuna,latitude,longitude,amount
0,2021,10,2,2021-10-06,0.0,Theft,Total Theft,0,0,VELEZ SARSFIELD,10.0,-34631129.0,-58489377.0,1
1,2021,10,3,2021-10-07,23.0,Robbery,Total Robbery,0,0,VELEZ SARSFIELD,10.0,-34624628.0,-58501039.0,1
2,2021,10,2,2021-10-06,12.0,Robbery,Total Robbery,1,0,MONTE CASTRO,10.0,-34623213.0,-58510132.0,1
3,2021,10,3,2021-10-07,20.0,Robbery,Total Robbery,0,0,VELEZ SARSFIELD,10.0,-34627172.0,-58498928.0,1
4,2021,10,5,2021-10-09,11.0,Theft,Total Theft,0,0,FLORESTA,10.0,-34626978.0,-58483313.0,1


## 10. Data Merging

In [347]:
merged_np = pd.merge(df_neighborhood, df_population, on='neighborhood', how='left')
merged_nps = pd.merge(merged_np, slums_count, on='neighborhood', how='left')
merged_npsh = pd.merge(merged_nps, df_hospitals, on='neighborhood', how='left')
merged_npsh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   neighborhood    48 non-null     object 
 1   commune         48 non-null     int64  
 2   perimeter_neib  48 non-null     float64
 3   area_neib       48 non-null     float64
 4   geometry        48 non-null     object 
 5   population      48 non-null     int64  
 6   slum            10 non-null     float64
 7   hospital_type   19 non-null     float64
dtypes: float64(4), int64(2), object(2)
memory usage: 3.1+ KB


In [384]:
merged_npshw = pd.merge(merged_npsh, gdf_subway_with_neigh, on='neighborhood', how='left')
merged_npshwns = pd.merge(merged_npshw, noise_inter_area_pivot, on='neighborhood', how='left')
merged_npshwns.head()

,neighborhood,commune,perimeter_neib,area_neib,geometry,population,slum,hospital_type,station,day_noise,night_noise
0,AGRONOMIA,15,6556.17,2122169.34,POLYGON ((-58.475888981732986 -34.591723461272...,13912,NaN,NaN,NaN,22.211583,20.322316
1,ALMAGRO,5,8537.90,4050752.25,POLYGON ((-58.416002854915654 -34.597854231564...,131699,NaN,1.0,5.0,28.458668,26.236751
2,BALVANERA,3,8375.82,4342280.27,POLYGON ((-58.392934155259674 -34.599636447011...,138926,NaN,1.0,12.0,27.401132,25.515022
3,BARRACAS,4,12844.17,7954579.06,POLYGON ((-58.3706620577617 -34.62949214687238...,89452,2.0,6.0,NaN,32.513541,30.952629
4,BELGRANO,13,20443.29,8025458.65,POLYGON ((-58.45056826999142 -34.5356114803969...,126267,NaN,1.0,2.0,26.384386,24.983233
